 Establish history of parking services in milwaukee?
- Duncan but Duncan got partially dropped? Portal still used though
- Route1 is the aplr contractor
- Route1 uses Genetic AutoVU 
    - https://resources.genetec.com/en-product-brochures/genetec-autovu-license-plate-recognition
        - Surprisingly low false positive rate?
    - Does Genetec manage all of the data?


In [ ]:
#!pip install pandarallel

In [ ]:
#!pip install plotly

In [ ]:
#!pip install kaleido #for PIL

In [ ]:
import PIL

In [ ]:
import io

In [ ]:
import pandas as pd

tickets = pd.read_csv("Tickets2012LatLngDropNa.csv")
tickets.head()

Below is v hacky and messy, but thats python! trying to get an understanding of the tickets given out across the city each day. Converting date to an int so I can plot the time series in one awful 3d mess. We can just use the date for most proccessing and other visualizations.

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
from pandarallel import pandarallel
pandarallel.initialize(nb_workers = 10, progress_bar=False)

def convert(date):
    return datetime.strptime(date, "%m/%d/%Y")

In [ ]:
tickettest_1 = tickets.head(25000)
tickettest_1["DATETIME"] = tickettest_1['ISSUEDATE'].parallel_apply(convert)

fig = plt.figure(figsize =(30,30))
ax = plt.axes(projection='3d')

zdata = tickettest_1["DATETIME"].values.astype("float64")
xdata = tickettest_1['Lat']
ydata = tickettest_1['Lng']
ax.scatter3D( xdata, zdata, ydata,);

Now were cooking with gas! The first goal is to build a some better maps with street overlay and also cluster. probs gonna go with fuzzy c means, and I would ideally rewrite this as a kotlin app, so I am looking for libraries that also have high levels of jvm support.

In [ ]:
import geopandas as gpd
import pandarallel
import matplotlib.pyplot as plt


gdf = gpd.GeoDataFrame(
    tickets, geometry=gpd.points_from_xy(tickets.Lng, tickets.Lat))

Lets plot our ticket data as a time series to see how things change over the year! This can inform us on what patterns may exist

In [ ]:
import plotly.express as px
token = "pk.eyJ1Ijoiam9obmVja2JlcmciLCJhIjoiY2xlbHJoMWViMHkweDNvbm1tZzA2NHA1cSJ9.IXat9PfPeqR3jd4uW4TETQ"
#saftey shmaftey


fig = px.scatter_mapbox( # yes i know lat and lon are flipped 
    tickets,
    lat="Lng",
    lon="Lat",
    hover_data=["ISSUETIME"],
    animation_frame="ISSUEDATE",
).update_layout(mapbox_style="streets",margin={"r":0,"t":0,"l":0,"b":0}, mapbox_accesstoken=token)
fig.show()

Lets quick export this as a gif

In [ ]:
# generate frames
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        "MilwaukeeParking2012Vis.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500,
        loop=0,
    )

Next lets do a heatmap over the course of the year.

In [ ]:
heat = px.density_mapbox(tickets, lat='Lng', lon='Lat' z= "", radius=10, zoom=0,
                        mapbox_style="street").update_layout(mapbox_accesstoken=token)
heat.show()

I gotta group by address then sum for the Z axis in the heat map, ill do that in a second
First, I want to quick discuss different ways to predict a ticket given a location.

1. A suggestion i recived was to first calculate the total number of cars that can be parked on each city block/ in front of each building, then use that to determine the probability on any given day. My concern with this is it assumes that every car parked is parked illegally, and has an equal oppurtunity to get ticketed.
2. I think a better approach may be to think less about cars and more about regions of the city/map. Say we split the city up into its respective blocks, then into the street segments that define the blocks. Then, understanding the geographical impliciation that if a parking enforcement officer scans one car on a a street segment, they must scan the rest of the cars on the street segment, we focus on the likelyhood of a officer passing through a given street segment.

- Theres this paper im readin that discusses a novel indicator, IPS or illegal parking score. How can we tune this for milwaukee? 
- the probability of a segment containing a parking ticket = likleyhood of your illegally parked car getting a  = (num of days segment gets a ticket/days in time frame)

Lets look at some streets. these were taken from Milwaukees open data portal.

In [ ]:
streets = gpd.read_file('street.zip')
streets.head()

We can defintely piece together street segments from this! First we can convert the linestring to points and then 

adverserial regression? rnn GAN? how can we seperate the data into routes? street data into street segment data -> cluster -> time series GAN? 

Or just turn time features into columns (ie month 0-11, day 0-6, ticket type 0-2). then it will all be discrete tabular data

Previous approaches:
- Previous approach in Madison/ deep learning paper NYC data
    - Note: RF performed great & they have provided some great insights into useful features and best practices for cleaning data
        - They have a tendency to not overfit 
    - https://www.tandfonline.com/doi/full/10.1080/19475683.2019.1679882?scroll=top&needAccess=true&role=tab
    - Regression for num of tickets given a location 
        - This might not really be useful for trying to decide a parking spot
    - Why is treating this at a simple supervised binary classification task silly
        - This leads to unbalanced and outright false data (they state 70 30 split, milwaukee will be hypothetically worse due to lower num of parking cops)
    - Overall good accuracy, “Given a search location and time for on-street parking, the mean testing error is less than 3 tickets for regression and the F1-score is 0.82 for parking legality prediction”
        - Can we do better?
        - Also can we perform other correlation testing to see how poverty & snow & alternate side plays into it?

    - DNN paper treats it as a time series prediction task  Thessaloniki, Greece data
        - https://cidl.csd.auth.gr/resources/conference_pdfs/Deep%20Learning%20for%20On-Street%20Parking%20Violation.pdf
        - Uses RNN
        - Predicts ticket percentage given time unit and street with pretty high accuracy, but the percentage is based on the capacity of the given street segment of the specific sector at a specific time slot t. Not something that is easy to calculate for milwaukee or a lot of cities
        - MAE of 0.169

How many cars on a street? Based on total number of cars parked OPEN DATA go back and stop and think about a car proven. 
- This assumes that every car entering the city is parked on the street. This is false
- But we could estimate number of cars on street with street length and mean car length + mean distance between cars (oh god I could gather that data by hand)



But what if we take a generative, rather than purely regressive approach to parking ticket detection. Applying generative modeling to a classification task
IDEA: GOOD SS-LEARNING MIGHT NEED A BAD GAN: https://arxiv.org/abs/1705.09783

- Its important to note, we have an awful class imbalance like 1:800 in the case of milwaukee and given our time granularity, instead, lets frame problem in the context of novelty detection/ one class classification with respect to the distribution of our features?
- A review of novelty detection (Pimento 2014)
        - https://romisatriawahono.net/lecture/rm/survey/machine%20learning/Pimentel%20-%20Novelty%20Detection%20-%202014.pdf
- A more recent (2021) review of OCC methods:
    - https://arxiv.org/pdf/2101.03064.pdf 
    - One class SVM: SVDD?
        - Doesn’t do well with high dimensional data? 
        - I would need to do heavy data processing, same with distance based methods
            - Distance based would be computationally expensive 
    - Adversarially learned one class classifier: (needs multi class but could work with imbalanced class?)
        -  https://arxiv.org/pdf/1802.09088v2.pdf
        - One network works as the novelty detector, while the other supports it by enhancing the inlier samples and distorting the outliers. The intuition is that the separability of the enhanced inliers and distorted outliers is much better than deciding on the original samples.
    - SSL GAN

        - Cherti et al. (2016) empirically demonstrated that a GAN that was trained with a regular loss function is able to generate out-of-class realistic images, e.g., a GAN trained on MNIST digits generated Latin letters
        - BIG IDEA: novelty detection based on generating novel examples given a feature matching loss function. If the generator is “estimating” the target distribution, how exactly does it do that? The conclusion is that only a bad generator, where pg (x) dne pdata(x) and which generates samples outside of the high-density areas of the real data distribution, is able to improve semi-supervised learning (Salimans et al. (2016)) what about unsupervised?? we want a mixture generator that will generate novel data distributed in nearby surrounding or at low-density regions of the true data manifold. 
            - Instead of directly maximizing the output of the discriminator, the new objective requires the generator to generate data that matches the statistics of the real data, where we use the discriminator only to specify the statistics that we think are worth matching. (Isn’t this what ARFs already do?when picking samples based on coverage? That’s why they don’t perform as well?) Specifically, we train the generator to match the expected value of the features on an intermediate layer of the discriminator. 
                - This is a natural choice of statistics for the generator to match, since by training the discriminator we ask it to find those features that are most discriminative of real data versus data generated by the current model. Specifically, we train the generator to match the expected value of the features on an intermediate layer of the discriminator. This is a natural choice of statistics for the generator to match, since by training the discriminator we ask it to find those features that are most discriminative of real data versus data generated by the current model.
                    - (How close is this to choosing the “most relevant pure leaves and sampling from marginals? the generator’s sampling strategy drives original and synthetic data closer together by proof. Prove that this is true for ARFS fancy sampling scheme? Given a 
        - Gan for novelty detection: 
            - If during training the generator generates a mixture of known data and novel data, the discriminator learns to discriminate novel data from known data and essentially becomes a novelty detector. its proved in that paper that in this case the discriminator become an optimal novelty detector (for a given false positive rate) 
            - in other words the mixture generator is a generator that generates a mixture of true data distribution pata(x) and some other data pother (x), where at least part of the pother (x) probability mass is concentrated in lower-density regions of pdata(x).
            - https://arxiv.org/pdf/1802.10560.pdf
    - ARFs 
        - can we prove the tree sampling scheme is a mixture generator? can we graph the densities of real and fake and see how they overlap?
        - The “generator” is a simple sampling scheme that draws from the marginals in adaptively selected subregions; the “discriminator” is a RF classifier. 
        - https://arxiv.org/pdf/2205.09435.pdf
    - Deep Gan method for density estimation: can we change the loss function? Have others?
        - https://www.pnas.org/doi/epdf/10.1073/pnas.2101344118
    - CGAN for tabular data: can you connect this the the adversarially learned one class classifier? 
        - https://arxiv.org/pdf/1907.00503.pdf